# 01. Ingest Data

Fetches raw data from Yahoo Finance and stores it using DataManager.

In [1]:
import sys
import os
import pandas as pd
from datetime import datetime, timedelta
from datetime import datetime, timedelta
import yfinance as yf
import re

# Add project root to path
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_path)
from utils.data_manager import DataManager
from dotenv import load_dotenv

load_dotenv()

# Constants
TICKER = "EURUSD=X"
YEARS = 5


## Ingestion Logic

In [2]:
# Initialize DataManager
dm = DataManager(data_type='raw', local_dir=os.path.join(root_path, 'data/raw'))
print(f"🚀 Starting ingestion (Environment: {dm.get_environment()})")

# Check freshness
if dm.is_data_current(max_age_hours=24):
    df = dm.get_latest_data()
    if len(df) > 250:
        print("✅ Data is already up-to-date. Skipping download.")
        print(f"📊 Rows: {len(df)}, Last Date: {df.index.max()}")
    else:
        print("⚠️ Data fresh but insufficient. forcing update.")
        
    # Fix: Clean any duplicates in loaded data
    # Fix: Clean any duplicates in loaded data
    if not df.empty:
        # Tuple-String Cleanup
        new_cols = []
        for c in df.columns:
            c = str(c)
            if (c.startswith("('") or c.startswith('("')) and "," in c:
                clean = c.strip("()").replace("'", "").replace('"', "").split(",")[0].strip()
                new_cols.append(clean)
            else:
                new_cols.append(c)
        df.columns = new_cols
        
        # Deduplicate
        df = df.loc[:, ~df.columns.duplicated()]
else:
    print("Data needs update.")


🚀 Starting ingestion (Environment: cloud)
Data needs update.


In [3]:
# Fetch Logic (Simulating the script)
latest_date = dm.get_latest_date()
end_date = datetime.now()

if latest_date and len(dm.get_latest_data()) > 250:
    start_date = latest_date + timedelta(days=1)
    if start_date.date() >= end_date.date():
        print("✅ Data is up to date.")
    else:
        print(f"📅 Updating from {start_date.date()}")
        # Download new
        new_df = yf.download(TICKER, start=start_date, end=end_date, interval="1d")
        if not new_df.empty:
            print(f"Fetched {len(new_df)} new rows.")
            
            # Fix: Flatten MultiIndex columns if present (common in new yfinance)
            if isinstance(new_df.columns, pd.MultiIndex):
                new_df.columns = new_df.columns.get_level_values(0)
            
            existing_df = dm.get_latest_data()
            if not existing_df.empty:
                # Tuple-String Cleanup for Existing Data
                new_cols = []
                for c in existing_df.columns:
                    c = str(c)
                    if (c.startswith("('") or c.startswith('("')) and "," in c:
                        clean = c.strip("()").replace("'", "").replace('"', "").split(",")[0].strip()
                        new_cols.append(clean)
                    else:
                        new_cols.append(c)
                existing_df.columns = new_cols
                existing_df = existing_df.loc[:, ~existing_df.columns.duplicated()]
                
            full_df = pd.concat([existing_df, new_df])
            full_df = full_df[~full_df.index.duplicated(keep='last')].sort_index()
            dm.save_data(full_df, metadata={'source': 'notebook_ingest'})
            print("💾 Saved updated data.")
        else:
            print("No new data found.")
else:
    # Full backfill
    start_date = end_date - timedelta(days=YEARS*365)
    print(f"📅 Full backfill from {start_date.date()}")
    df = yf.download(TICKER, start=start_date, end=end_date, interval="1d")
    if not df.empty:
        # Fix: Flatten MultiIndex columns
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
            
        dm.save_data(df, metadata={'source': 'notebook_ingest'})
        print(f"💾 Saved {len(df)} rows.")


📅 Updating from 2025-12-25


/var/folders/tx/45hhw1yn48jfdqn2hmnhbhvw0000gn/T/ipykernel_65129/2083239254.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  new_df = yf.download(TICKER, start=start_date, end=end_date, interval="1d")
[*********************100%***********************]  1 of 1 completed


Fetched 1 new rows.
💾 Saved updated data.


## Validation

In [4]:
df_final = dm.get_latest_data()
print(f"Total Rows: {len(df_final)}")
display(df_final.head())
display(df_final.tail())


Total Rows: 1301


,Close,High,Low,Open,Volume
Date,,,,,
2020-12-28,1.220510,1.225200,1.218353,1.220703,0
2020-12-29,1.222345,1.227599,1.222001,1.222001,0
2020-12-30,1.225295,1.230875,1.225205,1.225325,0
2020-12-31,1.229990,1.230951,1.222300,1.229906,0
2021-01-01,1.218027,1.221699,1.213499,1.217285,0


,Close,High,Low,Open,Volume
Date,,,,,
2025-12-19,1.172567,1.173750,1.170412,1.172539,0
2025-12-22,1.170809,1.177024,1.171084,1.171084,0
2025-12-23,1.176595,1.180359,1.176429,1.176443,0
2025-12-24,1.179551,1.180916,1.177426,1.179565,0
2025-12-26,1.177995,1.179941,1.176471,1.178689,0
